In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import momepy as mm
import networkx as nx
from shapely import Point, LineString, reverse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from pyproj import Transformer
import datetime
import math

In [2]:
import functions as ft
import corrections as ct
from importlib import reload
reload(ft)
reload(ct)

#print the current date into the info.txt file
current_date = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
ft.print_info(f"date: {current_date}")

date: 25-04-2023 00:40:39



## 1. CREACIO DEL SubGraph

Lectura geografica de la xarxa del 2021, creacio del Graph.

Definim subGraph com la xarxa connectada a la depuradora.

- no hi ha cap node/arc desconectat en el subGraph.

- estem extraien tambe els sistemes de drenatge urba no connectats a la xarxa residual. 

Cal comprovar: 

- nodes sanitaris fora del subgraph?


In [3]:
# 1.  Introduim la xarxa completa en un Graph de NetworkX
node = gpd.read_file("data/networks/AdG_node.gpkg")
arc = gpd.read_file("data/networks/AdG_arc.gpkg")

# 1.1 creem un graph amb la xarxa
G = nx.DiGraph()
G.add_nodes_from([(row['node_id'], row) for index, row in node.iterrows() if row['state']!=0])
G.add_edges_from([(row['node_1'], row['node_2'], row) for index, row in arc.iterrows() if row['state']!=0])
# estem eliminant els obsolets escollint nomes els state=0 (OPERATIVO)

print('La xarxa OPERATIVA completa de 2021 conte: ')
print(f"nodes =  {len(G.nodes)}")
print(f"arcs = {len(G.edges)}")

#2. Extraiem la sub-xarxa connectada a la depuradora
#2.1 Definim la depuradora
edar_code= 'PR0005300'
edar,data_edar=[(node,data) for node, data in G.nodes(data=True) if data['code'] == edar_code][0]
data_edar['epa_type']="OUTFALL"
data_edar['sys_type']="OUTFALL"
data_edar['node_type']="OUTFALL"

#2.2 definim el subGraph associat:
G_notDi = nx.Graph(G)
subGraph = nx.DiGraph(nx.subgraph(G,nx.node_connected_component(G_notDi, edar)))

print('La xarxa connectada a la depuradora conte: ')
print(f"nodes =  {len(subGraph.nodes)}")
print(f"arcs = {len(subGraph.edges)}")

La xarxa OPERATIVA completa de 2021 conte: 
nodes =  13572
arcs = 13682
La xarxa connectada a la depuradora conte: 
nodes =  12017
arcs = 12516


In [4]:
#3. Definim els elements de la nostra xarxa

# nodes marcats com OUTFALL
outfalls= [node for node, data in subGraph.nodes(data=True) if data['epa_type'] == 'OUTFALL']
print('nodes OUTFALL =', len(outfalls))

# nodes marcats com CHAMBERS
chambers= [node for node, data in subGraph.nodes(data=True) if data['nodecat_id'] == 'CEC' ]
print('nodes CHAMBER=', len(chambers))

# nodes marcats com CHAMBERS i son inici de xarxa
chambers_netinit= [node for node, data in subGraph.nodes(data=True) if data['nodecat_id'] == 'CEC' and subGraph.degree(node)==1]
print('nodes CHAMBER-NETINIT =', len(chambers_netinit))

# nodes marcats com MANHOLE
manholes= [node for node, data in subGraph.nodes(data=True) if data['sys_type'] == 'MANHOLE']
print('nodes MANHOLE =', len(manholes))

# arcs marcats com Sobreeixidor
sobreeixidors = [(node1, node2) for node1, node2, data in subGraph.edges(data=True) if data['category_type'] == 'Sobreeixidor']
print('arcs sobreeixidor (marcats) =', len(sobreeixidors))

sink = {node: data for node, data in subGraph.nodes(data=True) if subGraph.out_degree(node) == 0 and node!=edar and data['epa_type'] != 'OUTFALL'}
print('nodes direccionats cap a fora de la xarxa que no son outfall =', len(sink))

bad_chamber = {node: data for node, data in subGraph.nodes(data=True) if subGraph.degree(node) == 1 and node!=edar and data['nodecat_id'] =='CEC'}
print('camaras amb un sol arc:',len(bad_chamber))

nodes OUTFALL = 129
nodes CHAMBER= 189
nodes CHAMBER-NETINIT = 15
nodes MANHOLE = 9811
arcs sobreeixidor (marcats) = 278
nodes direccionats cap a fora de la xarxa que no son outfall = 37
camaras amb un sol arc: 15


## 2. CORRECCIÓ DE LES DIRECCIONS

1. mirem, per tots els nodes, si van direccionats cap a la depuradora o un outfall/chamberNETINIT

2. Mentre hi hagi nodes que no arribin a la depuradora ni a un outfall

- Per tots aquests nodes: Mirem si els seus veins arriben a la depuradora

- Si algun vei arriba a la depuradora, afegim un nou arc que va del node al vei i eliminem l'antic

- Si cap dels seus veins arriba a la depuradora, s'afegeix de nou a la llista  de nodes a revisar i es fa el loop

In [5]:
subGraph, malament, arcs_girats=ct.direccionament(subGraph,edar)


# REPORT: errors detectats en el direccionament
f = open("results/reports/report_direccions.txt", "w")
f.write(f"Nodes que no arriben a la depuradora = {len(malament)} \n")
for node in malament:
  f.write(str(subGraph.nodes[node]['code'])+",")
f.write(f"\n\nPer arreglar-los s'han canviat les direccions dels arcs = {len(arcs_girats)} \n")
for arc in arcs_girats:
  f.write(str(arc)+", ")
f.close()

nodes que no arriben a la depuradora = 891
nodes que no arriben a la depuradora ni a un outfall ni a un chamberNETINIT = 69
nodes que no arriben a la depuradora després del direccionament = 822
nodes que no arriben a la depuradora ni a un outfall després del direccionament = 1


# 3. Correccio de les dades

### code: 

code erronis de nodes (no dels arcs), els corretgim com PRd'node_id'

In [6]:
# detectem els nodes amb code erroni 
nodes_amb_codi_erroni_21 =([node for node, data in G.nodes(data=True) if data['code'][0:2] != 'PR'])
print('nodes amb codi erroni = ( ', len(nodes_amb_codi_erroni_21), ' )')

# corregim el code dels nodes amb identificadoe incorrectes, guardem node_id a link
nodes_amb_codi_erroni =([node for node, data in subGraph.nodes(data=True) if data['code'][0:2] != 'PR'])
codi_PRd=[]
for node in nodes_amb_codi_erroni:
  data = subGraph.nodes[node]
  data['code'] = 'PRd' + str(data['node_id'] )
  codi_PRd = 'PRd' + str(data['node_id'] )

for node, data in subGraph.nodes(data=True):
  data['link'] = str(data['node_id'] )


nodes amb codi erroni = (  2334  )


### top_elev

opcio a)

eliminem aquells valors que son nan o que estan fora del rang esperat (+-7 metres) del valor de altura obtingut pel raster geografic. 

omplim els valors desconeguts amb mitjana dels veins de primer ordre

opcio b) [escollida]

utilitzem la informacio geografica del raster

                qgis>toolbox>saple raster values




In [7]:
node_raster = gpd.read_file("data/subgraph_nodes_raster5m2020.gpkg")
nodes_erronis = []
jnan=0
jrang=0
for node, data in subGraph.nodes(data=True):
    z_raster=node_raster.loc[node_raster['code'] == data['code'], 'raster5m2020_1'].values[0]
    a=5
    if math.isnan(data['top_elev']):
        data['custom_top_elev'] = math.nan
        nodes_erronis.append(node)
        jnan=jnan+1
    if data['top_elev']>z_raster+a or data['top_elev']<z_raster-a:
        data['custom_top_elev'] = math.nan
        nodes_erronis.append(node)
        jrang=jrang+1
    else: 
        data['custom_top_elev'] = data['top_elev']

print(f"nodes amb top_elev fora del rang +-{a} metres = ({jrang}) i amb Nan ({jnan})")
print(f"nodes erronis inicials = {len(nodes_erronis)}")

# opcio de correccio a)  -------->  fem la mitjana dels veins de primer ordre
i = 0
while len(nodes_erronis) > 0:
    tmp = []
    for node_erroni in nodes_erronis:
        data_err = subGraph.nodes[node_erroni]
        first_neighbors = list(nx.all_neighbors(subGraph, node_erroni))
        costums_top_elev_neighbors = [data['custom_top_elev'] for node, data in subGraph.nodes(data=True) if node in first_neighbors and not math.isnan(data['custom_top_elev'])]
        if len(costums_top_elev_neighbors) > 0:
            data_err['custom_top_elev'] = np.mean(costums_top_elev_neighbors)
        else:
            tmp.append(node_erroni)
    i+=1
    nodes_erronis = tmp.copy()
    print('nodes_erronis a iteracio', i, ': ', len(nodes_erronis))


# opcio de correccio b) --------> fiquem custom_top_elev = z_raster
node_raster = gpd.read_file("data/subgraph_nodes_raster5m2020.gpkg")
for node, data in subGraph.nodes(data=True):
    z_raster=node_raster.loc[node_raster['code'] == data['code'], 'raster5m2020_1'].values[0]
    data['custom_top_elev'] = z_raster


nodes amb top_elev fora del rang +-5 metres = (98) i amb Nan (4405)
nodes erronis inicials = 4503
nodes_erronis a iteracio 1 :  1618
nodes_erronis a iteracio 2 :  846
nodes_erronis a iteracio 3 :  478
nodes_erronis a iteracio 4 :  265
nodes_erronis a iteracio 5 :  157
nodes_erronis a iteracio 6 :  91
nodes_erronis a iteracio 7 :  52
nodes_erronis a iteracio 8 :  27
nodes_erronis a iteracio 9 :  12
nodes_erronis a iteracio 10 :  4
nodes_erronis a iteracio 11 :  1
nodes_erronis a iteracio 12 :  0


### ymax

comprovem valors fora del rang esperat (1, 10) metres i nan

corrtegim amb mitjana dels veins

In [8]:
nodes_erronis = []
jnan=0
jrang=0
for node, data in subGraph.nodes(data=True):
    if math.isnan(data['ymax']):
        data['custom_ymax'] = math.nan
        nodes_erronis.append(node)
        jnan=jnan+1
    if data['ymax']<1 or data['ymax']>10:
        data['custom_ymax'] = math.nan
        nodes_erronis.append(node)
        jrang=jrang+1
    else: 
        data['custom_ymax'] = data['ymax']
print(f"nodes fora del rang (1,10) metres = ({jrang}) nodes amb Nan ({jnan})")
print('nodes erronis incials', len(nodes_erronis))

# fem la mitjana dels veins
i = 0
while len(nodes_erronis) > 0:
    tmp = []
    for node_erroni in nodes_erronis:
        data_err = subGraph.nodes[node_erroni]
        neighbors = list(nx.all_neighbors(subGraph, node_erroni))

        ymax_neighbors = [data['custom_ymax'] for node, data in subGraph.nodes(data=True) if node in neighbors and not math.isnan(data['custom_ymax'])]
        if len(ymax_neighbors) > 0:
            data_err['custom_ymax'] = np.mean(ymax_neighbors)
        else:
            tmp.append(node_erroni)
    i+=1
    nodes_erronis = tmp.copy()
    print('nodes_erronis a iteracio', i, ': ', len(nodes_erronis))

nodes fora del rang (1,10) metres = (1108) nodes amb Nan (2869)
nodes erronis incials 3977
nodes_erronis a iteracio 1 :  840
nodes_erronis a iteracio 2 :  243
nodes_erronis a iteracio 3 :  82
nodes_erronis a iteracio 4 :  32
nodes_erronis a iteracio 5 :  16
nodes_erronis a iteracio 6 :  8
nodes_erronis a iteracio 7 :  5
nodes_erronis a iteracio 8 :  4
nodes_erronis a iteracio 9 :  3
nodes_erronis a iteracio 10 :  2
nodes_erronis a iteracio 11 :  1
nodes_erronis a iteracio 12 :  0


# 4. CORRECIO DE LES SLOPES 

Definim la llista de bombes per excloure de la correcio

ALGORITHME DE SLOPES:

    1. per tots els arcs, calculem slopes, les figuem en uns llista

    2. si slope<0 de la llista: ajustem slope=0 i ajustem slopes dels veins (per actualitzar la llista)

    3. apliquem loop sobre la llista mentre quedin slope<zero_slope* o fins a max_iterations
        *(podem permetre un slope negatiu minim)

insertem els valors corregits a custom_ymax, custom_y1, custom_y2

In [9]:
import corrections as correct
from importlib import reload
reload(correct)


# Definim la list de bombes ( arc_id values to remove)
arc_bombes_list = ['TR0002408','TR0050421','TR0006306','TR0022007']
# Make a copy of the original subGraph
subGraph_sense_bombes = subGraph.copy()
# Remove the edges with arc_id values in the list
for node1, node2, data in subGraph.edges(data=True):
    if data['code'] in arc_bombes_list:
        edge_to_remove = (node1, node2)
        subGraph_sense_bombes.remove_edge(*edge_to_remove)
print('arcs amb bombes =', len(subGraph.edges))
print('arcs sense bombes =', len(subGraph_sense_bombes.edges))


# Compute the initial slopes
zero_slope=0 # min slope alowed
neg_slope=0
#slopes_ini=pd.DataFrame(columns=['code', 'slope_ini','Delta_h_ini'])
slopes_ini=[]
for node1, node2, data in subGraph_sense_bombes.edges(data=True):
    data_node1 = subGraph_sense_bombes.nodes[node1]
    data_node2 = subGraph_sense_bombes.nodes[node2]
    elev1=data_node1['custom_top_elev']-data_node1['custom_ymax']
    elev2=data_node2['custom_top_elev']-data_node2['custom_ymax']
    Delta_h=(elev1-elev2)
    slope=(elev1-elev2)/data['gis_length']
    #slopes_ini = pd.concat([slopes_ini, pd.DataFrame({'code': [data['code']], 'slope_ini': [slope],'Delta_h_ini': [Delta_h]})], ignore_index=True)
    slopes_ini.append({'code': data['code'], 'slope_ini': slope,'Delta_h_ini': Delta_h})
    if slope<=zero_slope:
        neg_slope=neg_slope+1
slopes_ini = pd.DataFrame.from_records(slopes_ini)
slopes_ini.sort_values(by='slope_ini')
print(f"slopes negatives inicials ( {neg_slope} ), amb min = ( {min(slopes_ini['slope_ini'])} )")


# ALGORITHME DE SLOPES
# 1. per tots els arcs, calculem slopes, les figuem en uns llista
# 2. si slope<0 de la llista: ajustem slope=0 i ajustem slopes dels veins (per actualitzar la llista)
# 3. apliquem loop sobre la llista mentre quedin slope<zero_slope* o fins a max_iterations
# *(podem permetre un slope negatiu minim)

print("---- Slope correction ----")
new_ymax_sense_bombes,new_y1_y2_sense_bombes, slopes_corrected=correct.slopes(subGraph_sense_bombes,zero_slope=zero_slope,max_iteratons=200)

# comparem els resultats
slope_merged = pd.merge(slopes_ini, slopes_corrected, on='code').sort_values(by='slope_ini')
print("---- Resultats ----")
#print(slope_merged.head(5))

# insertem els valors corregits a custom_ymax
for node, data in subGraph.nodes(data=True):
    code = data['code']
    new_ymax_value = new_ymax_sense_bombes.loc[new_ymax_sense_bombes['code'] == code, 'new_ymax'].item()
    data['custom_ymax'] = new_ymax_value
    # correcio dels ymax negatuys: per error en precissio del raster
    if data['custom_ymax']<0:
        data['custom_top_elev']= data['custom_top_elev']+2*abs(data['custom_ymax'])
        data['custom_ymax']=abs(data['custom_ymax'])


# ajustem la profunditat de la boca dels arcs al valor 
for node1, node2, data in subGraph.edges(data=True):
    if data['code'] in new_y1_y2_sense_bombes['code']:
        data['custom_y1'] = new_y1_y2_sense_bombes.loc[new_y1_y2_sense_bombes['code'] == data['code']]['new_y1'].item()
        data['custom_y2'] = new_y1_y2_sense_bombes.loc[new_y1_y2_sense_bombes['code'] == data['code']]['new_y2'].item()
    else:
        data1=subGraph.nodes[node1]
        data2=subGraph.nodes[node2]
        data['custom_y1'] = data1['custom_ymax']
        data['custom_y2'] = data2['custom_ymax']

arcs amb bombes = 12516
arcs sense bombes = 12512
slopes negatives inicials ( 3025 ), amb min = ( -1.6259273670337826 )
---- Slope correction ----
Iteració 0. Falten 3025 amb slope negatives amb min -1.6259273670337826
Iteració 1. Falten 1296 amb slope negatives amb min -1.1715690786102706
Iteració 2. Falten 1187 amb slope negatives amb min -0.6161962466769746
Iteració 3. Falten 1006 amb slope negatives amb min -0.3968009345443222
Iteració 4. Falten 833 amb slope negatives amb min -0.32967470721620823
Iteració 5. Falten 706 amb slope negatives amb min -0.2902719873486173
Iteració 6. Falten 630 amb slope negatives amb min -0.2591061810435728
Iteració 7. Falten 535 amb slope negatives amb min -0.23384635406817766
Iteració 8. Falten 483 amb slope negatives amb min -0.21248522495260191
Iteració 9. Falten 437 amb slope negatives amb min -0.19386063644892224
Iteració 10. Falten 394 amb slope negatives amb min -0.17731189188511412
Iteració 11. Falten 369 amb slope negatives amb min -0.1624179

In [19]:
slope_merged.sort_values(by='slope_final',ascending=False).head(10)
#pk Delta_h_corr dona constant?

,code,slope_ini,Delta_h_ini,slope_corr,Delta_h_corr,slope_final
1459,TR0025166,-0.253962,-0.159996,0.003349,0.00211,0.15
503,TR0004515,0.318674,0.289993,0.002319,0.00211,0.15
12227,TR0020842,0.000000,0.000000,0.002638,0.00211,0.15
495,TR0011484,0.820000,0.820000,0.002110,0.00211,0.15
4925,TR0000838,0.221354,0.212500,0.002198,0.00211,0.15
5503,TR0023098,0.603779,0.640005,0.001991,0.00211,0.15
12071,TR0020739,0.160376,0.169998,0.001991,0.00211,0.15
281,TR0011643,0.000000,0.000000,0.002293,0.00211,0.15
1939,TR0024643,0.836961,1.540009,0.001147,0.00211,0.15
557,TR0011299,0.000000,0.000000,0.002110,0.00211,0.15


## 5. AJUST DE SOBREEIXIDORS

definim arc de "sobreximent" com primer arc direccionats cap a un node que no es la depuradura.
    

In [11]:

# detectem arcs sobreeixidors
no_llegan_depuradora = [node for node in subGraph.nodes if not nx.has_path(subGraph, node, edar)]
node1_sobreeixidor = []
node2_sobreeixidor= []
for node in no_llegan_depuradora:
    veins=nx.all_neighbors(subGraph, node)
    for vei in veins:
        if nx.has_path(subGraph, vei, edar):
            node1_sobreeixidor.append(vei)
            node2_sobreeixidor.append(node)         
sobreeixidors_detected = [(node1, node2, data) for node1, node2, data in subGraph.edges(data=True) if  (data['node_1'] in node1_sobreeixidor) and (data['node_2'] in node2_sobreeixidor)]
sobreeixidors_marked = [(node1, node2, data) for node1, node2, data in subGraph.edges(data=True) if data['category_type'] == 'Sobreeixidor']

print('arcs detectats com sobreixidors = ',len(sobreeixidors_detected))
print('arcs marcats com sobreixidors = ',len(sobreeixidors_marked))


# corregim custom_y1 dels arcs sobreeixidors
sobreeixidors = sobreeixidors_detected
thresholds=[] #list de custom_y1 a modificar per afegir thresholds
b = 0.25  #altura extra
for node1, node2, data in sobreeixidors:
    in_edges = list(subGraph.in_edges(node1))
    out_edges = list(subGraph.out_edges(node1))
    out_edges = [(o, d) for o, d in out_edges if o!=node1 or d!=node2]

    # detectem la altura y_min del sobreiximent estimada (meitat del tub) i fem una llista de altures
    y_min = [] 
    for o, d in in_edges:
        data_in_edge = subGraph.edges[(o, d)]
        y_min.append(data_in_edge['custom_y2'] - data_in_edge['cat_geom1']/2)

    for o, d in out_edges:
        data_out_edge = subGraph.edges[(o, d)]
        y_min.append(data_out_edge['custom_y1'] - data_out_edge['cat_geom1']/2)
    
    
    data_node1 = subGraph.nodes[node1] # node sobreixidor
    threshold = np.min(y_min) - b  # ens quedem amb el valor mes petit de la llista i li substraiem b
    thresholds.append({'arc_code':data['code'],'custom_y1':data['custom_y1'],'custom_y1_threshold':threshold,'node1':data_node1['code'],'arcs_connect_to_node1':len(y_min)+1})
    subGraph.edges[(node1,node2)]['custom_y1'] = threshold  # update the value of custom_y1 with the threshold value
thresholds=pd.DataFrame.from_records(thresholds)


thresholds.sort_values(by='custom_y1_threshold',ascending=True).head(10)    
#thresholds[thresholds['arc_code'] == 'TR0003809']

arcs detectats com sobreixidors =  136
arcs marcats com sobreixidors =  278


,arc_code,custom_y1,custom_y1_threshold,node1,arcs_connect_to_node1
33,TR0050439,0.052499,-0.447501,PR0005319,3
68,TR0022828,0.647548,0.197548,PR0022033,3
83,TR0050138,0.813797,0.248797,PR0005630,2
86,TR0024982,0.839390,0.289390,PR0004440,5
0,TR0024972,1.900000,0.450000,PR0024351,3
127,TR0011149,0.922103,0.472103,PR0010728,3
4,TR0025822,0.940000,0.490000,PR0024353,3
71,TR0022590,0.980702,0.530702,PR0021841,2
65,TR0022839,0.935001,0.535001,PR0002665,4
3,TR0050500,0.991615,0.541615,PR0030062,3


In [12]:
for node1, node2, data in sobreeixidors:
    in_edges = list(subGraph.in_edges(node1))
    out_edges = list(subGraph.out_edges(node1))

    if data['code']=='TR0003809':
    
        print('code =', data['code'], 'custom_y1', data['custom_y1'], 'arc sobreixidor') 

    # detectem la altura dels tubs connectats al node 
    
        for o, d in in_edges:
            data_in_edge = subGraph.edges[(o, d)]
            print('code =', data_in_edge['code'], 'custom_y2', data_in_edge['custom_y2'])
        for o, d in out_edges:
            data_out_edge = subGraph.edges[(o, d)]
            print('code =', data_out_edge['code'], 'custom_y1', data_out_edge['custom_y1'])
    

code = TR0003809 custom_y1 2.0139420538036177 arc sobreixidor
code = TR0003758 custom_y2 3.2639420538036177
code = TR0003809 custom_y1 2.0139420538036177
code = TR0050332 custom_y1 3.2639420538036177
code = TR0003759 custom_y1 3.2639420538036177


In [13]:
# detectem els tubs que tenen y1+geom1 per sobre del terra i corregim

c=0.5
arc_y1_sobresortint=[]
arc_y2_sobresortint=[]
for node1, node2, data_arc in subGraph.edges(data=True):

    h1_arc=data_arc['custom_y1']-data_arc['cat_geom1']
    data_node1=subGraph.nodes[node1]
    in_edges1 = list(subGraph.in_edges(node1))
    out_edges1 = list(subGraph.out_edges(node1))
    if h1_arc<0:
        arc_y1_sobresortint.append({'arc_code':data['code'],'h1_arc':h1_arc})
        data_node1['custom_top_elev']=data_node1['custom_top_elev']+abs(h1_arc)+c
        data_node1['custom_ymax']= data_node1['custom_ymax']+abs(h1_arc)+c
        for o, d in in_edges1:
            data_in_edge1 = subGraph.edges[(o, d)]
            data_in_edge1['custom_y2']=data_in_edge1['custom_y2']+abs(h1_arc)+c
        for o, d in out_edges1:
            data_out_edge1 = subGraph.edges[(o, d)]
            data_out_edge1['custom_y1']=data_out_edge1['custom_y1']+abs(h1_arc)+c

    h2_arc=data_arc['custom_y2']-data_arc['cat_geom1']
    data_node2=subGraph.nodes[node2]
    in_edges2 = list(subGraph.in_edges(node2))
    out_edges2 = list(subGraph.out_edges(node2))
    if h2_arc<0:
        arc_y2_sobresortint.append({'arc_code':data['code'],'h2_arc':h2_arc})
        data_node2['custom_top_elev']=data_node2['custom_top_elev']+abs(h2_arc)+c
        data_node2['custom_ymax']= data_node2['custom_ymax']+abs(h2_arc)+c
        for o, d in in_edges2:
            data_in_edge2 = subGraph.edges[(o, d)]
            data_in_edge2['custom_y2']=data_in_edge2['custom_y2']+abs(h2_arc)+c
        for o, d in out_edges2:
            data_out_edge2 = subGraph.edges[(o, d)]
            data_out_edge2['custom_y1']=data_out_edge2['custom_y1']+abs(h2_arc)+c

print("arcs corregits que sobresurtien del terra per y1 = ", len(arc_y1_sobresortint))
print("arcs corregits que sobresurtien del terra per y2 = ", len(arc_y2_sobresortint))



arcs corregits que sobresurtien del terra per y1 =  47
arcs corregits que sobresurtien del terra per y2 =  18


#### REPORT: errors detectats sobreeixidors

In [14]:

#els nodes de sobreiximent detectats  haurien de coiniceixin amb els marcats

# els nodes que no arriben a la depuradora
no_llegan_depuradora = [node for node in subGraph.nodes if not nx.has_path(subGraph, node, edar)]
# identifiquem els veins dels que no arriven i mirem si algun dells arriba a la depuradora 
# el vei que arribi sera el node 'pare' del sobreixidor
nodes_sobreeixidors = []
for node in no_llegan_depuradora:
    veins=nx.all_neighbors(subGraph, node)
    for vei in veins:
        if nx.has_path(subGraph, vei, edar):
            nodes_sobreeixidors.append(node)

print('nodes sobreeixidor detectats =',len(nodes_sobreeixidors))   


arcs_sobreeixidors_marcats = [(node1, node2, data) for node1, node2, data in subGraph.edges(data=True) if data['category_type'] == 'Sobreeixidor']
#print('arcs sobreeixidor marcats =',len(arcs_sobreeixidors_marcats))  

nodes_sobreeixidors_marcats = [node2 for node1, node2, data in arcs_sobreeixidors_marcats]
print('nodes sobreeixidor marcats =',len(nodes_sobreeixidors_marcats))

nodes_marcat_sobreeixidors_erronis=list(set(nodes_sobreeixidors_marcats)-set(nodes_sobreeixidors))

nodes_sobreeixidors_no_marcats=list(set(nodes_sobreeixidors)-set(nodes_sobreeixidors_marcats))


print('nodes marcats com a sobreeixidors que no ho son:', len(nodes_marcat_sobreeixidors_erronis))
print('nodes sobreeixidors detectat que no estan marcats:',len(nodes_sobreeixidors_no_marcats))

nodes sobreeixidor detectats = 136
nodes sobreeixidor marcats = 278
nodes marcats com a sobreeixidors que no ho son: 166
nodes sobreeixidors detectat que no estan marcats: 26


In [15]:

# imprimim el report
f = open("results/reports/report_sobreeixidors.txt", "w")

f.write(f'nodes sobreeixidor detectats = {len(nodes_sobreeixidors)}\n')   
for node in nodes_sobreeixidors:
  f.write(str(subGraph.nodes[node]['code'])+",")

f.write(f'\n\nnodes marcats com a sobreeixidors que no ho son = {len(nodes_marcat_sobreeixidors_erronis)}\n')   
for node in nodes_marcat_sobreeixidors_erronis:
  f.write(str(subGraph.nodes[node]['code'])+",")

f.write(f'\n\nnodes sobreeixidors detectat que no estan marcats = {len(nodes_sobreeixidors_no_marcats)}\n')   
for node in nodes_sobreeixidors_no_marcats:
  f.write(str(subGraph.nodes[node]['code'])+",")

f.close()

## 6. IMPRIMIR EL SUBGRAF CONNECTAT A LA DEPURADORA

1. diccionary     node_id_new,   node_id_old,   code: we  make a join between giswater_nodes and node with commun value code
- giswater_nodes has an atribute called node_id that will be renamed as node_id_new 
- node has an atribute called node_id that will be renamed as node_id_old

2. impresio de .gpkg

In [16]:
#1.  canviem la nomenclatura de node_id en els nodes i node1, node2 dels arcs
# eliminem informacio inecessaria

# Load the GeoDataFrames
giswater_nodes=ft.download("AdG_2021", "subgraph.node")
# Rename the columns
giswater_nodes = giswater_nodes.rename(columns={'node_id': 'node_id_new'})
# Create a dictionary mapping codes to node IDs in giswater_nodes
code_to_node_id = dict(zip(giswater_nodes['code'], giswater_nodes['node_id_new']))

# use subgraph data to creat diccionary
dic_nodes = {}  
# Update dic_nodes with node_id_new information
for node, data in subGraph.nodes(data=True):
    node_id = data['node_id']
    code = data['code']
    node_id_new = code_to_node_id.get(code)
    if node_id_new is not None:
        dic_nodes[node] = {'node_id': node_id, 'node_id_new': node_id_new, 'code': code}
    else:
        print(f"No node with code '{code}' in giswater_nodes")


# update the node_id information in the subGraph 
for node, data in subGraph.nodes(data=True):
    # Get the 'node_id_new' value from the dictionary 'dic_nodes' using the 'code' attribute of the node
    node_info = dic_nodes.get(node)
    if node_info is not None:
        data['node_id'] = node_info['node_id_new']
    else:
        print(f"No information found for node '{node}'")

    # Set the value to NaN
    data['elev'] = np.nan
    data['custom_elev'] = np.nan
    data['sys_top_elev'] = np.nan
    data['sys_ymax'] = np.nan
    data['sys_elev'] = np.nan


# update the arc data information in the subGraph 
for node1, node2, data in subGraph.edges(data=True):
    # Get the 'node_id_new' value for 'node1' and 'node2' from the dictionary 'dic_nodes' using the 'code' attribute of the nodes
    node1_info = dic_nodes.get(node1)
    node2_info = dic_nodes.get(node2)
    if node1_info is not None and node2_info is not None:
        data['node_1'] = node1_info['node_id_new']
        data['node_2'] = node2_info['node_id_new']
    else:
        print(f"No information found for arc '{node1} - {node2}'")
    
    
    # Set the value to NaN

    data['r1'] = np.nan
    data['r2']=np.nan
    data['z1']=np.nan
    data['z2']=np.nan

    data['custom_elev1']=np.nan
    data['custom_elev2']=np.nan

    data['sys_elev1']=np.nan
    data['sys_elev2']=np.nan

    data['sys_y1']=np.nan
    data['sys_y2']=np.nan

    data['elev1']
    data['elev2']

    data['slope']=np.nan
    data['custom_length']=np.nan


print('done!')

done!


In [17]:
#2. imprimim el graph en un .gpkg per visualitzar en QGIS

#transformem les coordenades
transformer = Transformer.from_crs("EPSG:25831", "WGS84")

#afegim els nodes
for index, row in subGraph.nodes(data=True):
    if row['state'] != 0:
        x = row['geometry'].x
        y = row['geometry'].y

        long, lat = transformer.transform(x, y)
        row['longitude'] = long
        row['latitude'] = lat
        row['x'] = x
        row['y'] = y
        
        row['geometry'] = (lat, long)    

# fem una copia del subgraph i el reindexem amb la geometria dels nodes
subGraph_copy = subGraph.copy()
geometry_nodes = {node: data['geometry'] for node, data in subGraph_copy.nodes(data=True)}
subGraph_copy = nx.relabel_nodes(subGraph_copy, geometry_nodes)

# guardem el subgraph en dos .gpkg, un per nodes i un per arcs
points, lines = mm.nx_to_gdf(subGraph_copy)
points.set_geometry("geometry")
points.to_file("results/networks/nodes_corretgits_v2.gpkg", layer='nodes', driver="GPKG")
lines.set_geometry("geometry")

geometry= {}
for u, v, data in G.edges(data=True):
    geometry[u, v] = LineString((G.nodes[u]["geometry"],G.nodes[v]["geometry"]))
    nx.set_edge_attributes(G, geometry, "geometry")

lines.to_file("results/networks/arcs_corretgits_v2.gpkg", layer='arcs', driver="GPKG")

c:\Users\ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\momepy\utils.py:430: UserWarning: Approach is not set. Defaulting to 'primal'.
  warnings.warn("Approach is not set. Defaulting to 'primal'.")
